In [5]:
import requests
import csv
from datetime import timezone, datetime
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
import os

# constant definitions
API_KEY=os.getenv('TORN_API_KEY_ENV')
FILE_NAME = "output.csv"
MASTER_FILE_NAME = "all_attacks.csv"
LOST = 'lost'
DEFENDER_NAME = 'defender_name'
DEFENDER_ID = 'defender_id'
GAINED = 'gained'
RESPECT = 'respect'
CHAIN_BONUS = 'chain_bonus'
MODIFIERS = 'modifiers'
ATTACKER_NAME = 'attacker_name'
ATTACKER_ID = 'attacker_id'
DEFENDER_FACTION = 'defender_faction'
DEFENDER_FACTION_NAME = 'defender_factionname'
ATTACKER_FACTION_NAME = 'attacker_factionname'
RANKED_WAR = 'ranked_war'
TIMESTAMP_STARTED = 'timestamp_started'
ATTACKS_KEY = 'attacks'
OUTSIDE_ATTACKS_KEY = 'outside_attacks'
NAME_KEY= 'name'
MODIFIER_KEY="modifiers"
RESPECT_GAINED_KEY="gained"
RESPECT_LOST_KEY="lost"
SELECTED_ATTACK_KEY=[
    'code',
    'timestamp_started',
    'timestamp_ended',
    'attacker_id',
    'attacker_name',
    'attacker_faction',
    'attacker_factionname',
    'defender_id',
    'defender_name',
    'defender_faction',
    'defender_factionname',
    'result',
    'stealthed',
    'respect',
    'chain',
    'raid',
    'ranked_war',
    'respect_gain',
    'respect_loss',
    'modifiers',
]
MODIFIER_ATTRBS=[
    'fair_fight',
    'war',
    'retaliation',
    'group_attack',
    'overseas',
    'chain_bonus'
]
# all attacks
attacks_master_list=[]

HEADER = ['id', 'name', 'attacks', 'outside_attacks', 'gained', 'lost']

extra_calls = [',applications', ',armor', ',armorynews', ',attacknews',
               ',basic', ',boosters', ',cesium', ',chain', ',chainreport',
               ',chains', ',contributors', ',crimenews', ',crimes',
               ',currency', ',donations', ',drugs', ',fundsnews',
               ',mainnews', ',medical', ',membershipnews', ',positions',
               ',reports', ',revives', ',revivesfull', ',stats', ',temporary',
               ',territory', ',territorynews', ',timestamp', ',upgrades', ',weapons '
               ]

def visualize_report():
    # Create a dataframe from the data
    df = pd.read_csv('output.csv')

    # Sort the dataframe by the "attacks" column
    df = df.sort_values(by='attacks', ascending=False)
    plt.figure(figsize=(12,6))
    # bars = plt.bar(df['name'], df['attacks'], df['lost'], df['outside_attacks'])

    # Create a bar chart of the "attacks" column
    plt.bar(df['name'], df['attacks'], width=0.5)
    plt.xticks(rotation=90)
    plt.xlabel('Name')
    plt.ylabel('Attacks')
    plt.title('Attacks by Name')

    # Get the bleeders
    bleeders = df[df['lost'] > df['gained']].sort_values(by='lost', ascending=False)

    # Get names and lost values
    names = bleeders['name']
    lost = bleeders['lost']

    # Create the bar graph
    plt.figure(figsize=(12, 6))
    plt.bar(names, lost, width=0.5)
    plt.xticks(rotation=90)
    plt.xlabel('Name')
    plt.ylabel('Lost')
    plt.title('Bleeders')
    plt.show()

def get_start_time_end_time(text):
    # Extract start and end times
    start_str, end_str = text.split(" until ")
    start_time = datetime.strptime(start_str, "%H:%M:%S - %d/%m/%y")
    end_time = datetime.strptime(end_str, "%H:%M:%S - %d/%m/%y")

    # Convert start and end times to epoch time
    start_time_epoch = int(start_time.timestamp())
    end_time_epoch = int(end_time.timestamp())
    return {start_time_epoch, end_time_epoch}
    

def is_ranked_war_attack(attack_obj):
    return attack_obj[RANKED_WAR] == 1

def get_timestamp(start_day, start_hour, start_min, month, year):
    return int(datetime(year, month, start_day, start_hour, start_min, tzinfo=timezone.utc).timestamp())

def save_summary():
    with open(FILE_NAME, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(HEADER)
        for member in scores:
            writer.writerow([member, scores[member][NAME_KEY],scores[member][ATTACKS_KEY],
                            scores[member][OUTSIDE_ATTACKS_KEY],scores[member][RESPECT_GAINED_KEY],scores[member][RESPECT_LOST_KEY]])

def load_list():
    try:
        with open("all_attacks.csv", "r") as file:
            reader = csv.reader(file, delimiter=",")
            my_list = list(reader)
            print("loaded from previously cached file")
            return my_list
    except FileNotFoundError:
        print("all_attacks.csv not found")
        return None
    

def save_list(list):
    print(f"total number of attacks (in + out)-> {len(list)}")
    with open(MASTER_FILE_NAME, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        headers = SELECTED_ATTACK_KEY
        headers.remove(MODIFIER_KEY)
        headers.extend(MODIFIER_ATTRBS)
        writer.writerow(headers)
        for attack in list:
            writer.writerow([
                # main attributes
                attack[SELECTED_ATTACK_KEY[0]],attack[SELECTED_ATTACK_KEY[1]],attack[SELECTED_ATTACK_KEY[2]],
                attack[SELECTED_ATTACK_KEY[3]],attack[SELECTED_ATTACK_KEY[4]],attack[SELECTED_ATTACK_KEY[5]],
                attack[SELECTED_ATTACK_KEY[6]],attack[SELECTED_ATTACK_KEY[7]],attack[SELECTED_ATTACK_KEY[8]],
                attack[SELECTED_ATTACK_KEY[9]],attack[SELECTED_ATTACK_KEY[10]],attack[SELECTED_ATTACK_KEY[11]],
                attack[SELECTED_ATTACK_KEY[12]],attack[SELECTED_ATTACK_KEY[13]],attack[SELECTED_ATTACK_KEY[14]],
                attack[SELECTED_ATTACK_KEY[15]],attack[SELECTED_ATTACK_KEY[16]],attack[SELECTED_ATTACK_KEY[17]],
                attack[SELECTED_ATTACK_KEY[18]],
                # modifiers                                                                                                       
                attack[MODIFIER_KEY][MODIFIER_ATTRBS[0]],attack[MODIFIER_KEY][MODIFIER_ATTRBS[1]],
                attack[MODIFIER_KEY][MODIFIER_ATTRBS[2]],attack[MODIFIER_KEY][MODIFIER_ATTRBS[3]],
                attack[MODIFIER_KEY][MODIFIER_ATTRBS[4]],attack[MODIFIER_KEY][MODIFIER_ATTRBS[5]]
            ])    

def process_attacks(hero_faction_name, enemy_faction_name, master_list):
    is_header=False
    scores={}
    for attack_obj in master_list:
        if is_header is False:
            is_header = True
            continue
        # outside hits
        if attack_obj[ATTACKER_FACTION_NAME] == hero_faction_name and attack_obj[DEFENDER_FACTION_NAME] != enemy_faction_name:
            # create a new record if not encountered before
            if attack_obj[ATTACKER_ID] not in scores.keys():
                scores[attack_obj[ATTACKER_ID]] = {'name': attack_obj[ATTACKER_NAME],'attacks': 0,'outside_attacks': 0,'gained': 0,'lost': 0}    
            # update existing record
            scores[attack_obj[ATTACKER_ID]][OUTSIDE_ATTACKS_KEY] = scores[attack_obj[ATTACKER_ID]][OUTSIDE_ATTACKS_KEY] + 1
            outgoing = outgoing + 1

        is_rw_attack = is_ranked_war_attack(attack_obj)

        # attack is against enemy faction and made by hero faction
        if attack_obj[DEFENDER_FACTION_NAME] == enemy_faction_name and is_rw_attack:
            # create a new record if not encountered before
            if attack_obj[ATTACKER_ID] not in scores.keys():
                scores[attack_obj[ATTACKER_ID]] = winning_member_record(attack_id, attacks_list)
                # update existing record
                if attack_obj[MODIFIERS][CHAIN_BONUS] < 10:
                    scores[attack_obj[ATTACKER_ID]][GAINED] = scores[attack_obj[ATTACKER_ID]][GAINED] + attack_obj[RESPECT]
                    scores[attack_obj[ATTACKER_ID]][ATTACKS_KEY] = scores[attack_obj[ATTACKER_ID]][ATTACKS_KEY] + 1
                    outgoing = outgoing + 1

        # attack is against hero faction and made by target enemy faction
        if attack_obj[DEFENDER_FACTION_NAME] == hero_faction_name and is_rw_attack:
            # create a new record if not encountered before
            if attack_obj[DEFENDER_ID] not in scores.keys():
                scores[attack_obj[DEFENDER_ID]] = losing_member_record(attack_id, attacks_list)
            # update existing record
            if attack_obj[MODIFIERS][CHAIN_BONUS] < 10:
                scores[attack_obj[DEFENDER_ID]][LOST] = scores[attack_obj[DEFENDER_ID]][LOST] + attack_obj[RESPECT]
                incoming = incoming + 1 

def fetch_attacks(end_stamp, start_stamp, api_key):
    next_call_stamp = start_stamp
    processed_attack_records = []
    count = 0
    outgoing = 0
    incoming = 0
    print('fetching attacks using API ...')

    while next_call_stamp <= end_stamp:
        count += 1
        time.sleep(2)
        # print(f'{next_call_stamp}')
        request = requests.get(
            f'https://api.torn.com/faction/?selections=attacks'
            f'{extra_calls[count % len(extra_calls)]}&from={next_call_stamp}&key={api_key}')

        response_body = request.json()
        # attack object found
        try:
            attacks_list = response_body[ATTACKS_KEY]

            for attack_id in attacks_list:
                # update timestamp to set the next 'from' field in the API call
                attack_obj = attacks_list[attack_id]
                if attack_obj[TIMESTAMP_STARTED] > next_call_stamp:
                    next_call_stamp = attack_obj[TIMESTAMP_STARTED] - 1

                if attack_id not in processed_attack_records:
                    processed_attack_records.append(attack_id)
                    #is_rw_attack = is_ranked_war_attack(attack_id, attacks_list)
                    # prepare a master list of all attacks
                    attacks_master_list.append({key: attack_obj[key] for key in SELECTED_ATTACK_KEY if key in attack_obj})

        except KeyError:
            print("No more attacks left to process")
            break

    print(f'Total hits made: {outgoing}')
    print(f'Total hits received: {incoming}')
    print(f'Total API calls made -> {count}')
    return attacks_master_list

def generate_report(hero_faction_name, enemy_faction_name, api_key):
    if api_key is not None:
        masked_api_key = "*****" + api_key[-5:]
        print("Using API Key:", masked_api_key)
    else:
        print("API Key not defined.")

    start_stamp, end_stamp = get_start_time_end_time("15:00:00 - 07/04/23 until 13:03:05 - 08/04/23")
    print(f"start_stamp:{start_stamp} - end_stamp:{end_stamp}")
    # step 1 - call torn api and get a list of all the attacks & received during the RW period OR load from previously written file
    attacks_master_list = load_list()
    if attacks_master_list is None:
        attacks_master_list = fetch_attacks(end_stamp, start_stamp, api_key)
        save_list(attacks_master_list)
    # step 2 - process attacks
    process_attacks(hero_faction_name, enemy_faction_name, attacks_master_list)


generate_report(hero_faction_name="The Power Rangers", enemy_faction_name="Drunk Squad", api_key=API_KEY)
#visualize_report()


Using API Key: *****UyVgD
start_stamp:1680865200 - end_stamp:1680944585
loaded from previously cached file


TypeError: list indices must be integers or slices, not str